In [1]:
import pandas as pd
import csv

C:\Users\l3n3c\AppData\Local\Temp\ipykernel_47876\4056139722.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
#creation of SQLite database
import sqlite3

conn = sqlite3.connect('etl_project.db')
cursor = conn.cursor()

In [13]:
# creating tables
cursor.execute("CREATE TABLE IF NOT EXISTS songs (song text primary key, day integer, month integer, year integer, streams integer)")

cursor.execute("CREATE TABLE IF NOT EXISTS artists_songs (artist_name text not null, song text primary key, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS lyrics (song text primary key, lyrics text not null, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS charts (song text primary key, spotify integer, apple integer, deezer integer, shazam integer, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS concerts (artist_name text, event text primary key, location text, venue text, CONSTRAINT fk_artist FOREIGN KEY (artist_name) REFERENCES artist_songs(artist_name))")

In [21]:
# extracting from csv 1
# csv columns: track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
csv_file = 'spotify-2023.csv'
with open(csv_file, 'r') as csv_data:
    csv_reader = csv.DictReader(csv_data)
    for row in csv_reader:
        # inserting into songs table
        cursor.execute('''
            INSERT OR IGNORE INTO songs (song, day, month, year, streams)
            VALUES (?, ?, ?, ?, ?)
        ''', (row['track_name'], int(row['released_day']), int(row['released_month']), int(row['released_year']), row['streams']))
        # inserting into charts table
        cursor.execute('''
            INSERT OR IGNORE INTO charts (song, spotify, apple, deezer, shazam)
            VALUES (?, ?, ?, ?, ?)
        ''', (row['track_name'], row['in_spotify_charts'], row['in_apple_charts'], row['in_deezer_charts'], row['in_shazam_charts']))
        # inserting into artists_songs
        if int(row['artist_count']) == 1:
            cursor.execute('''
                INSERT OR IGNORE INTO artists_songs (artist_name, song)
                VALUES (?, ?)
            ''', (row['artist(s)_name'], row['track_name']))

conn.commit()

In [24]:
# extracting from csv 2
# csv info: artist,song,link,text
csv_file2 = 'spotify_millsongdata.csv'
with open(csv_file2, 'r') as csv_data2:
    csv_reader2 = csv.DictReader(csv_data2)
    for row in csv_reader2:
        # inserting into songs table
        cursor.execute('''
            INSERT OR IGNORE INTO lyrics (song, lyrics)
            VALUES (?, ?)
        ''', (row['song'], row['text']))

conn.commit()

In [2]:
# getting list of artists
cursor.execute("SELECT DISTINCT artist_name FROM artists_songs")
artists = [row[0] for row in cursor.fetchall()]

print(len(artists))

301


In [8]:
# calling from api
import requests


# url = 'https://api.seatgeek.com/2/events?client_id=MYCLIENTID'
client = 'NDA4Mzg3OTB8MTcxMjUwODIwOC40MzI4MDg'

for artist in artists:
    slug = artist.lower().replace(" ", "-")
    url = 'https://api.seatgeek.com/2/events?performers[primary].slug=' + slug + '&client_id=' + client
    response = requests.get(url)
    json_data = response.json()
    # print(json_data)
    if response.status_code == 200 and json_data['meta']['total'] != 0:
        #inserting into db
        # db info: artist_name text, event text primary key, location text, venue text, CONSTRAINT fk_artist FOREIGN KEY (artist_name) REFERENCES artist_songs(artist_name)
        # json example: {'events': [], 'meta': {'total': 0, 'took': 2, 'page': 1, 'per_page': 10, 'geolocation': None}}
        # events: (on seatgeek website)
        
        for event in json_data['events']:
            cursor.execute('''
            INSERT OR IGNORE INTO concerts (artist_name, event, location, venue)
            VALUES (?, ?, ?, ?)
        ''', (artist, event['title'], event['venue']['city'], event['venue']['name']))
conn.commit()

In [20]:
#SQL Queries to demonstrate
# selecting songs from a certain artist (table = artists_songs)
artist = "Olivia Rodrigo"
cursor.execute("SELECT song FROM artists_songs WHERE artist_name = '"+ artist + "';")
rows = cursor.fetchall() 
for row in rows:
    print(row)

# counting how many concerts each artist has
cursor.execute("SELECT artist_name, COUNT() FROM concerts GROUP BY artist_name;")
rows = cursor.fetchall() 
for row in rows:
    print("Artist: " + row[0] + ", Number of concerts: " + str(row[1]))

# selecting all concerts where a certain song might be played
mysong = "vampire"
cursor.execute("SELECT event FROM concerts JOIN artists_songs ON concerts.artist_name = artists_songs.artist_name WHERE artists_songs.song = '"+ mysong + "';")
rows = cursor.fetchall() 
for row in rows:
    print(row)


('vampire',)
('good 4 u',)
('drivers license',)
('deja vu',)
('happier',)
('favorite crime',)
('jealousy, jealousy',)
Artist: 50 Cent, Number of concerts: 1
Artist: Adele, Number of concerts: 1
Artist: Aerosmith, Number of concerts: 9
Artist: Bad Bunny, Number of concerts: 1
Artist: Beach Weather, Number of concerts: 2
Artist: Benson Boone, Number of concerts: 2
Artist: BoyWithUke, Number of concerts: 2
Artist: Bruno Mars, Number of concerts: 1
Artist: Chris Brown, Number of concerts: 2
Artist: Cigarettes After Sex, Number of concerts: 1
Artist: Conan Gray, Number of concerts: 1
Artist: Drake, Number of concerts: 2
Artist: ENHYPEN, Number of concerts: 1
Artist: Ed Sheeran, Number of concerts: 1
Artist: Feid, Number of concerts: 2
Artist: Freddie Dredd, Number of concerts: 1
Artist: Fujii Kaze, Number of concerts: 1
Artist: Glass Animals, Number of concerts: 1
Artist: Gunna, Number of concerts: 1
Artist: Hozier, Number of concerts: 1
Artist: James Arthur, Number of concerts: 2
Artist: J

In [10]:
conn.commit()
conn.close()